In [1]:
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips
from realign import transform_skeleton_final


In [21]:
plate = 26

dates_datetime = get_dates_datetime(directory,plate)


In [22]:
len(dates_datetime)

120

In [29]:
# from directory import directory, run_parallel
directory = "/projects/0/einf914/data/"

plate = 26
begin = 30
end = 72
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_pruned_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)


In [27]:
def get_time(dates_datetimes,i,j):
    seconds = (dates_datetimes[j]-dates_datetimes[i]).total_seconds()
    return(seconds/3600)

In [30]:
from IPython.display import clear_output
start=0
finish = end-begin
for i in range(start,finish):
    plt.close('all')
    clear_output(wait=True)
    plot_t_tp1([], [], None, None, skels[i], ims[i], save=f'Figure/im{i}',time=f't = {int(get_time(dates_datetime_chosen,0,i))}h')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
start=0
finish = end-begin
img_array = []
for t in range(start,finish):
    img = cv2.imread(f'Figure/im{t}.png')
    img_array.append(img)
imageio.mimsave(f'Figure/movie{plate}temp_{dates[start]}_{dates[finish]}.gif', img_array,duration = 1)

In [2]:
plate = 33
begin = 0
end = 20
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'/scratch/shared/mrozemul/Fiji.app/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [3]:
def get_im_stitched(exp,boundaries,t):
    i = t
    plate = exp.plate
    directory = "/scratch/shared/mrozemul/Fiji.app/" 
    listdir=os.listdir(directory) 
    dates = exp.dates
    date =dates [i]
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
    try:
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
    except:
        print('error_name')
        path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
    skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')
    Rot= skel['R']
    trans = skel['t']
    shape = (3000,4096)
    xs =[c[0] for c in tileconfig[2]]
    ys =[c[1] for c in tileconfig[2]]
    dim = (boundaries[3]-boundaries[2],boundaries[1]-boundaries[0])
    ims = []
    for name in tileconfig[0]:
    #     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
        ims.append(imageio.imread(f'{name}'))
    im_stitched = np.zeros(dim,dtype=np.uint8)
    for index,im in enumerate(ims):
        xyangle = np.array([int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))])
    #     xyangle_corrected =  np.transpose(np.transpose(np.dot(Rot,xyangle))+trans)
        xyangle_corrected =  xyangle
        limits = int(xyangle_corrected[0]-boundaries[0]),int(xyangle_corrected[1]-boundaries[2])
        if limits[1]>=-shape[0] and limits[1]<=dim[0] and limits[0]>=-shape[1] and limits[0]<=dim[1]:
            b_im_s = max(0,limits[1]),min(dim[0],limits[1]+shape[0]),max(0,limits[0]),min(dim[1],limits[0]+shape[1])
            shape_crop = b_im_s[1]-b_im_s[0], b_im_s[3]-b_im_s[2]
            b_im0 =-limits[1] if limits[1]<0 else 0
            b_im1 = b_im0+shape_crop[0]
            b_im2 = -limits[0] if limits[0]<0 else 0
            b_im3 = b_im2+shape_crop[1]
            b_im = b_im0,b_im1,b_im2,b_im3
            im_stitched[b_im_s[0]:b_im_s[1],b_im_s[2]:b_im_s[3]] = im[b_im[0]:b_im[1],b_im[2]:b_im[3]]
    return(im_stitched)

In [4]:
def transform_skeleton_final_for_show(skeleton_doc,Rot,trans):
    skeleton_transformed={}
    transformed_keys = np.round(np.transpose(np.dot(Rot,np.transpose(np.array(list(skeleton_doc.keys())))))+trans).astype(np.int)
    i=0
    for pixel in list(transformed_keys):
        i+=1
        skeleton_transformed[(pixel[0],pixel[1])]=1
    skeleton_transformed_sparse=sparse.lil_matrix((27000, 50000))
    for pixel in list(skeleton_transformed.keys()):
        i+=1
        skeleton_transformed_sparse[(pixel[0],pixel[1])]=1
    return(skeleton_transformed_sparse)

In [5]:
def get_skeleton(exp,boundaries,t):
    i = t
    plate = exp.plate
    directory = "/scratch/shared/mrozemul/Fiji.app/" 
    listdir=os.listdir(directory) 
    dates = exp.dates
    date =dates [i]
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
    try:
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
    except:
        print('error_name')
        path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
        tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
    skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')
    skelet = skel['skeleton']
    skelet = sparse_to_doc(skelet)
    Rot= skel['R']
    trans = skel['t']
    skel_aligned = transform_skeleton_final_for_show(skelet,np.linalg.inv(Rot),(-np.dot(np.linalg.inv(Rot),trans)))
    output = skel_aligned[boundaries[2]:boundaries[3],boundaries[0]:boundaries[1]].todense()
    kernel = np.ones((5,5),np.uint8)
    output = cv2.dilate(output.astype(np.uint8),kernel,iterations = 2)
    return(output)

In [115]:
plt.close('all')
t=1
window = (25000,30000,6000,11000)
skelet = get_skeleton(exp,window,t)
im_stitched = get_im_stitched(exp,window,t)
nodes = [node.label for node in exp.nodes if t in node.ts() and node.degree(t) ==1 and node.pos(t)[1]>=window[0] and node.pos(t)[1]<=window[1] and node.pos(t)[0]>=window[2] and node.pos(t)[0]<=window[3]]

In [116]:
tips = [node.label for node in exp.nodes if t in node.ts() and node.degree(t) ==1 and node.pos(t)[1]>=window[0] and node.pos(t)[1]<=window[1] and node.pos(t)[0]>=window[2] and node.pos(t)[0]<=window[3]]

In [117]:
junction =  [node.label for node in exp.nodes if t in node.ts() and node.degree(t) >=2 and node.pos(t)[1]>=window[0] and node.pos(t)[1]<=window[1] and node.pos(t)[0]>=window[2] and node.pos(t)[0]<=window[3]]

In [118]:
plot_t_tp1([],list(np.random.choice(tips,100)),exp.positions[t],exp.positions[t],skelet,im_stitched,shift=(window[2],window[0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.close('all')
for t in range(0,20):
    window = (25000,30000,6000,11000)
    skelet = get_skeleton(exp,window,t)
    im_stitched = get_im_stitched(exp,window,t)
    tips = [node.label for node in exp.nodes if t in node.ts() and node.degree(t) ==1 and node.pos(t)[1]>=window[0] and node.pos(t)[1]<=window[1] and node.pos(t)[0]>=window[2] and node.pos(t)[0]<=window[3]]
    junction =  [node.label for node in exp.nodes if t in node.ts() and node.degree(t) >=2 and node.pos(t)[1]>=window[0] and node.pos(t)[1]<=window[1] and node.pos(t)[0]>=window[2] and node.pos(t)[0]<=window[3]]
    plot_t_tp1(junction,tips,exp.positions[t],exp.positions[t],skelet,im_stitched,shift=(window[2],window[0]),save=f'Data/im{t}',time=f't={4*t}')

In [14]:
im_stitched = get_im_stitched(exp,window,15)

In [ ]:
img_array = []
for t in range(0,13):
    img = cv2.imread(f'Data/im{t}.png')
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
imageio.mimsave('Data/movie.mp4', img_array,duration=1)

In [12]:

img_array = []
for t in range(0,15):
    img = cv2.imread(f'Data/im{t}.png')
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('Data/project.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 1, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [85]:
plt.close('all')
plot_t_tp12(junction,tips,exp.positions[t],exp.positions[t],skelet,im_stitched,shift=(window[2],window[0]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
def plot_t_tp1(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1, Rot = np.array([[1,0],[0,1]]),trans = np.array([0,0]),relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None):
    right = 0.90
    top = 0.90
    R = np.linalg.inv(Rot)
    tr = -np.dot(np.linalg.inv(Rot),trans)
    if len(save)>=1:
        fig=plt.figure(figsize=(14,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    ax.text(right, top, time,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax.transAxes,color='white',fontsize=20)
    for node in node_list_t:
        position = np.dot(R,pos_t[node])+tr
        t = ax.text((position[1]-shift[1])//compress, (position[0]-shift[0])//compress, str(relabel_t(node)), ha="center", va="center",
                    size=size,
                    bbox=bbox_props1)
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            position = np.dot(R,pos_tp1[node])+tr
            t = ax.text((position[1]-shift[1])//compress, (position[0]-shift[0])//compress, str(relabel_tp1(node)), ha="center", va="center",
                        size=size,
                        bbox=bbox_props2)
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()

In [69]:
def plot_t_tp12(node_list_t,node_list_tp1,pos_t,pos_tp1,imt,imtp1, Rot = np.array([[1,0],[0,1]]),trans = np.array([0,0]),relabel_t=lambda x:x,relabel_tp1=lambda x:x, shift=(0,0),compress=1,save='',time=None):
    right = 0.90
    top = 0.90
    R = np.linalg.inv(Rot)
    tr = -np.dot(np.linalg.inv(Rot),trans)
    if len(save)>=1:
        fig=plt.figure(figsize=(14,12))
        size = 10
    else:
        fig = plt.figure()
        size = 5
    ax = fig.add_subplot(111)
    ax.imshow(imtp1, cmap='gray',interpolation='none')
    ax.imshow(imt, cmap='jet', alpha=0.5,interpolation='none')
    bbox_props1 = dict(boxstyle="circle", fc="grey")
    bbox_props2 = dict(boxstyle="circle", fc="white")
    ax.text(right, top, time,
        horizontalalignment='right',
        verticalalignment='bottom',
        transform=ax.transAxes,color='white')
    for node in node_list_t:
        position = np.dot(R,pos_t[node])+tr
        plt.scatter((position[1]-shift[1])//compress, (position[0]-shift[0])//compress,s=2,c='black')
    for node in node_list_tp1:
        if node in pos_tp1.keys():
            position = np.dot(R,pos_tp1[node])+tr
            plt.scatter((position[1]-shift[1])//compress, (position[0]-shift[0])//compress,s=2,c='green')
    if len(save)>=1:
        plt.savefig(save)
        plt.close(fig)
    else:
        plt.show()

In [51]:
plot_t_tp1([],[],None,None,skelet,im_stitched)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im_stitched,cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
hyph_chose = [hyph for hyph in exp.hyphaes if hyph.end.label == 7016]

In [29]:
hyph = hyph_chose[0]
hyph.ts

[14, 15, 16, 17, 18, 19]

In [31]:
hyph.end.ts()

[5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [17]:
hyph = choice(exp.hyphaes)
hyph.ts

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [30]:
plt.close('all')
exp.plot([hyph.ts[0],hyph.ts[-1]],[[hyph.root.label,hyph.end.label]]*2)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.close('all')
hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
t=9

In [58]:
hyph.root.pos(t)

array([14396, 19883])

In [14]:
boundaries = (10000,20000,7000,15000)
t = 10
i = t
plate = exp.plate
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
dates = exp.dates
date =dates [i]
directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')
Rot= skel['R']
trans = skel['t']
shape = (3000,4096)
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (boundaries[1]-boundaries[0],boundaries[3]-boundaries[2])
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(imageio.imread(f'{name}'))
im_stitched = np.zeros(dim,dtype=np.uint8)
for index,im in enumerate(ims):
    xyangle = np.array([int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))])
#     xyangle_corrected =  np.transpose(np.transpose(np.dot(Rot,xyangle))+trans)
    xyangle_corrected =  xyangle
    limits = int(xyangle_corrected[0]-boundaries[0]),int(xyangle_corrected[1]-boundaries[2])
    if limits[1]>=-shape[0] and limits[1]<=dim[0] and limits[0]>=-shape[1] and limits[0]<=dim[1]:
        b_im_s = max(0,limits[1]),min(dim[0],limits[1]+shape[0]),max(0,limits[0]),min(dim[1],limits[0]+shape[1])
        shape_crop = b_im_s[1]-b_im_s[0], b_im_s[3]-b_im_s[2]
        b_im0 =-limits[1] if limits[1]<0 else 0
        b_im1 = b_im0+shape_crop[0]
        b_im2 = -limits[0] if limits[0]<0 else 0
        b_im3 = b_im2+shape_crop[1]
        b_im = b_im0,b_im1,b_im2,b_im3
        im_stitched[b_im_s[0]:b_im_s[1],b_im_s[2]:b_im_s[3]] = im[b_im[0]:b_im[1],b_im[2]:b_im[3]]
plt.close('all')
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(im_stitched)

(0, 470, 0, 921)
(0, 538, 0, 4033)
(0, 611, 3159, 7255)
(0, 686, 6370, 8000)
(0, 2882, 0, 742)
(0, 2951, 0, 3981)
(22, 3022, 3106, 7202)
(94, 3094, 6317, 8000)
(2303, 5303, 0, 685)
(2371, 5371, 0, 3926)
(2443, 5443, 3051, 7147)
(2517, 5517, 6262, 8000)
(4711, 7711, 0, 629)
(4783, 7783, 0, 3866)
(4853, 7853, 2990, 7086)
(4925, 7925, 6200, 8000)
(7121, 10000, 0, 577)
(7194, 10000, 0, 3812)
(7264, 10000, 2935, 7031)
(7334, 10000, 6144, 8000)
(9526, 10000, 0, 519)
(9596, 10000, 0, 3753)
(9665, 10000, 2873, 6969)
(9739, 10000, 6083, 8000)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
boundaries = (13000,20000,18000,28000)
t = 10
i = t
plate = exp.plate
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
dates = exp.dates
date =dates [i]
directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
skel = read_mat(path_snap+'/Analysis/skeleton_realigned.mat')
Rot= skel['R']
trans = skel['t']
shape = (3000,4096)
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (boundaries[1]-boundaries[0],boundaries[3]-boundaries[2])
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(imageio.imread(f'{name}'))
im_stitched = np.zeros(dim,dtype=np.uint8)



(0, 2008, 0, 693)
(0, 2082, 0, 3909)
(0, 2152, 3020, 7000)
(0, 2221, 6244, 7000)


In [26]:
im.shape

(3000, 4096)

In [46]:
b_im

(2008, 5008, 693, 4789)

In [23]:
dim

(7000, 10000)

In [7]:
max(0,limits[0]),limits[0]+shape[1],limits[0]

(3020, 7116, 3020)

In [9]:
im[-min(0,limits[1]):,-min(0,limits[0]):].shape

(2152, 4096)

In [11]:
max(0,limits[0]),limits[0]+shape[1]

(3020, 7116)